In [1]:
import pandas as pd

In [2]:
df = pd.read_pickle('data/tft_games_pickled.csv')

print(df.shape)
df.head()

(76676, 9)


,level,placement,length,round,queue,lobby_rank,augments,traits,units
0,10,2,36:33,7-1,Ranked,Bronze I,"[theroadlesstraveled, jeweledlotus, capricious...","[{'name': 'sage', 'count': 5}, {'name': 'drago...","[{'name': 'azir', 'star': 1, 'items': ['infini..."
1,10,1,35:17,6-4,Ranked,Bronze I,"[teambuilding, radiantrefactor, fatedcrown]","[{'name': 'fated', 'count': 1}, {'name': 'behe...","[{'name': 'ahri', 'star': 3, 'items': ['spearo..."
2,7,8,25:43,4-6,Ranked,Silver IV,"[buriedtreasuresii, mythiccrown, balancedbudget+]","[{'name': 'mythic', 'count': 5}, {'name': 'arc...","[{'name': 'bard', 'star': 1, 'items': [None]},..."
3,7,5,33:43,6-1,Ranked,Bronze I,"[blisteringstrikes, biggrabbag, pandorasitemsii]","[{'name': 'storyweaver', 'count': 5}, {'name':...","[{'name': 'galio', 'star': 2, 'items': ['redem..."
4,8,6,27:38,5-3,Ranked,N/A,"[pandorasitems, teambuilding, bruisercrest]","[{'name': 'bruiser', 'count': 6}, {'name': 'st...","[{'name': 'aatrox', 'star': 2, 'items': [None]..."


In [3]:
if df.isnull().values.any():
    print("There are {} null values in the df".format(df.isnull().sum().sum()))
    nan_rows = df[df.isnull().T.any()].to_string()
    # From this, we see that the null values are mostly from lobby_rank, as all players in the
    # game are unranked. Since we have a lot of data points, we can drop any rows that have any nans/null values
    df = df.dropna(how='any',axis=0)
else:
    print("There are no null values in the dataframe.")
    
df

There are no null values in the dataframe.


,level,placement,length,round,queue,lobby_rank,augments,traits,units
0,10,2,36:33,7-1,Ranked,Bronze I,"[theroadlesstraveled, jeweledlotus, capricious...","[{'name': 'sage', 'count': 5}, {'name': 'drago...","[{'name': 'azir', 'star': 1, 'items': ['infini..."
1,10,1,35:17,6-4,Ranked,Bronze I,"[teambuilding, radiantrefactor, fatedcrown]","[{'name': 'fated', 'count': 1}, {'name': 'behe...","[{'name': 'ahri', 'star': 3, 'items': ['spearo..."
2,7,8,25:43,4-6,Ranked,Silver IV,"[buriedtreasuresii, mythiccrown, balancedbudget+]","[{'name': 'mythic', 'count': 5}, {'name': 'arc...","[{'name': 'bard', 'star': 1, 'items': [None]},..."
3,7,5,33:43,6-1,Ranked,Bronze I,"[blisteringstrikes, biggrabbag, pandorasitemsii]","[{'name': 'storyweaver', 'count': 5}, {'name':...","[{'name': 'galio', 'star': 2, 'items': ['redem..."
4,8,6,27:38,5-3,Ranked,N/A,"[pandorasitems, teambuilding, bruisercrest]","[{'name': 'bruiser', 'count': 6}, {'name': 'st...","[{'name': 'aatrox', 'star': 2, 'items': [None]..."
...,...,...,...,...,...,...,...,...,...
76671,8,2,36:35,7-2,Ranked,Silver I,"[toomuchcandy, ghostlycrown, rollthedice]","[{'name': 'ghostly', 'count': 6}, {'name': 'in...","[{'name': 'aatrox', 'star': 3, 'items': ['fist..."
76672,7,8,23:33,4-4,Ranked,Silver II,"[epoch, teambuilding, finalascension]","[{'name': 'mythic', 'count': 3}, {'name': 'sto...","[{'name': 'bard', 'star': 2, 'items': ['hextec..."
76673,9,3,36:09,6-6,Ranked,Silver I,"[buriedtreasuresiii, gargantuanresolve, rollth...","[{'name': 'umbral', 'count': 6}, {'name': 'beh...","[{'name': 'alune', 'star': 2, 'items': ['jewel..."
76674,8,6,33:26,5-6,Ranked,Silver II,"[escortquest, heroicgrabbag, stationarysupportii]","[{'name': 'storyweaver', 'count': 3}, {'name':...","[{'name': 'bard', 'star': 3, 'items': ['redbuf..."


Flattening Unit Column

In [4]:
# Flatten unit list into separate columns for each unit
units = pd.DataFrame(df.units.to_numpy().tolist())

# Fill none units from the dataframe with pre-defined missing unit dictionary
none_dict = {'name': 'None', 'star': 0, 'items': [None, None, None]}
units_df = units.where(units.notna(), lambda x: none_dict)

labels_units = []
# Function to flatten dictionaries in a DataFrame column
def flatten_column_units(input_df, column):
    df_normalized = pd.json_normalize(input_df[column])
    item_columns = [f"item_{i+1}" for i in range(df_normalized['items'].apply(len).max())]
    df_items = pd.DataFrame(df_normalized['items'].tolist(), columns=item_columns)
    df_normalized = pd.concat([df_normalized, df_items.iloc[:,0:3]], axis=1).drop(columns=['items'])
    df_normalized.columns = [f"unit_{column}_{sub_column}" for sub_column in df_normalized.columns]
    labels_units.extend([f"unit_{column}_name", f"unit_{column}_item_1", f"unit_{column}_item_2", f"unit_{column}_item_3"])
    return df_normalized

# Iterate over columns in the DataFrame and flatten dictionaries
flattened_dfs_units = [flatten_column_units(units_df, column) for column in units_df.columns]

# Concatenate flattened DataFrames along columns
flattened_df_units = pd.concat(flattened_dfs_units, axis=1)

flattened_df_units.head()

,unit_0_name,unit_0_star,unit_0_item_1,unit_0_item_2,unit_0_item_3,unit_1_name,unit_1_star,unit_1_item_1,unit_1_item_2,unit_1_item_3,...,unit_12_name,unit_12_star,unit_12_item_1,unit_12_item_2,unit_12_item_3,unit_13_name,unit_13_star,unit_13_item_1,unit_13_item_2,unit_13_item_3
0,azir,1,infinityforce,manazane,blacksmithsgloves,diana,2,None,None,None,...,None,0,None,None,None,None,0,None,None,None
1,ahri,3,spearofshojin,jeweledgauntlet,giantslayer,aphelios,1,None,None,None,...,None,0,None,None,None,None,0,None,None,None
2,bard,1,None,None,None,caitlyn,3,guinsoosrageblade,guardbreaker,None,...,None,0,None,None,None,None,0,None,None,None
3,galio,2,redemption,warmogsarmor,steadfastheart,garen,2,None,None,None,...,None,0,None,None,None,None,0,None,None,None
4,aatrox,2,None,None,None,bard,2,guinsoosrageblade,hextechgunblade,giantslayer,...,None,0,None,None,None,None,0,None,None,None


Flatten Traits Column

In [5]:
labels_traits = []
def flatten_column_traits(input_df, column):
    df_normalized = pd.json_normalize(input_df[column])
    df_normalized.columns = [f"trait_{column}_{sub_column}" for sub_column in df_normalized.columns]
    labels_traits.extend([f"trait_{column}_name"])
    return df_normalized

# Traits preprocessing
traits = pd.DataFrame(df.traits.to_numpy().tolist())
none_dict_traits = {'name': 'None', 'count': 0}
traits_df = traits.where(traits.notna(), lambda x: none_dict_traits)
print(traits_df.head().to_string())

flattened_dfs_traits = [flatten_column_traits(traits_df, column) for column in traits_df.columns]

# Concatenate flattened DataFrames along columns
flattened_df_traits = pd.concat(flattened_dfs_traits, axis=1)

flattened_df_traits.head()

                                     0                                    1                                  2                                 3                                 4
0         {'name': 'sage', 'count': 5}   {'name': 'dragonlord', 'count': 3}   {'name': 'altruist', 'count': 2}  {'name': 'behemoth', 'count': 2}     {'name': 'dryad', 'count': 2}
1        {'name': 'fated', 'count': 1}     {'name': 'behemoth', 'count': 4}   {'name': 'arcanist', 'count': 2}     {'name': 'dryad', 'count': 2}   {'name': 'duelist', 'count': 2}
2       {'name': 'mythic', 'count': 5}     {'name': 'arcanist', 'count': 2}   {'name': 'behemoth', 'count': 2}   {'name': 'ghostly', 'count': 2}  {'name': 'heavenly', 'count': 2}
3  {'name': 'storyweaver', 'count': 5}     {'name': 'altruist', 'count': 2}   {'name': 'arcanist', 'count': 2}   {'name': 'bruiser', 'count': 2}   {'name': 'ghostly', 'count': 2}
4      {'name': 'bruiser', 'count': 6}  {'name': 'storyweaver', 'count': 3}  {'name': 'trickshot', 'count

,trait_0_name,trait_0_count,trait_1_name,trait_1_count,trait_2_name,trait_2_count,trait_3_name,trait_3_count,trait_4_name,trait_4_count
0,sage,5,dragonlord,3,altruist,2,behemoth,2,dryad,2
1,fated,1,behemoth,4,arcanist,2,dryad,2,duelist,2
2,mythic,5,arcanist,2,behemoth,2,ghostly,2,heavenly,2
3,storyweaver,5,altruist,2,arcanist,2,bruiser,2,ghostly,2
4,bruiser,6,storyweaver,3,trickshot,2,great,1,None,0


Flatten Augments Column

In [6]:
labels_augments = ['augment_1', 'augment_2', 'augment_3']
flattened_df_augments = pd.DataFrame(df.augments.to_numpy().tolist(), columns=labels_augments)

flattened_df_augments.head()

,augment_1,augment_2,augment_3
0,theroadlesstraveled,jeweledlotus,capriciousforge
1,teambuilding,radiantrefactor,fatedcrown
2,buriedtreasuresii,mythiccrown,balancedbudget+
3,blisteringstrikes,biggrabbag,pandorasitemsii
4,pandorasitems,teambuilding,bruisercrest


Merge Column DataFrames into Final Dataframe for Model

In [7]:
# Function to convert length column into decimal time
def convert_to_decimal(time_str):
    minutes, seconds = map(int, time_str.split(':'))
    return minutes + seconds / 60.0

# Function to convert stage column to number of rounds played
def convert_to_stage(time_str):
    rounds, stages = map(int, time_str.split('-'))
    return rounds * 7 + stages

final_df = pd.concat([df, flattened_df_augments, flattened_df_traits, flattened_df_units], axis=1).drop(['augments', 'traits', 'units'], axis=1)

final_df['length'] = final_df['length'].apply(convert_to_decimal)
#
final_df['round'] = final_df['round'].apply(convert_to_stage)
labels = final_df.select_dtypes('object').columns.tolist()


final_df.head()

,level,placement,length,round,queue,lobby_rank,augment_1,augment_2,augment_3,trait_0_name,...,unit_12_name,unit_12_star,unit_12_item_1,unit_12_item_2,unit_12_item_3,unit_13_name,unit_13_star,unit_13_item_1,unit_13_item_2,unit_13_item_3
0,10,2,36.550000,50,Ranked,Bronze I,theroadlesstraveled,jeweledlotus,capriciousforge,sage,...,None,0,None,None,None,None,0,None,None,None
1,10,1,35.283333,46,Ranked,Bronze I,teambuilding,radiantrefactor,fatedcrown,fated,...,None,0,None,None,None,None,0,None,None,None
2,7,8,25.716667,34,Ranked,Silver IV,buriedtreasuresii,mythiccrown,balancedbudget+,mythic,...,None,0,None,None,None,None,0,None,None,None
3,7,5,33.716667,43,Ranked,Bronze I,blisteringstrikes,biggrabbag,pandorasitemsii,storyweaver,...,None,0,None,None,None,None,0,None,None,None
4,8,6,27.633333,38,Ranked,N/A,pandorasitems,teambuilding,bruisercrest,bruiser,...,None,0,None,None,None,None,0,None,None,None


Save Dataframe to CSV 

In [8]:
file_name = ["data/tft_games_dp.csv", "data/tft_games_pickled_dp.csv"]

try:
    final_df.to_csv(file_name[0])
    final_df.to_pickle(file_name[1])
    print("File {}, {} saved".format(file_name[0], file_name[1]))
except Exception as e:
    print("File {} could not be saved. Error {}".format(file_name, e))

File data/tft_games_dp.csv, data/tft_games_pickled_dp.csv saved


On to the next step! Exploratory Data Analysis!

Continued in exploratory_data_analysis.ipynb ->